# Single dish (GBT) conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.48 already installed.


## Download dataset

https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse/unittest/tclean/venus_ephem_test.ms

In [2]:
import toolviper

toolviper.utils.data.download(file="sdimaging.ms")

[2025-04-02 12:49:30,953]  WARNING    viperlog:  File exists: /Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/toolviper/utils/data/.dropbox 
[2025-04-02 12:49:30,953]     INFO    viperlog:  Updating file metadata information ...  
 

 Download List  
 ─────────────── 
  sdimaging.ms

[2025-04-02 12:49:32,013]     INFO    viperlog:  File exists: sdimaging.ms 


## Convert to Processing Set

In [3]:
from xradio.measurement_set import convert_msv2_to_processing_set

ms_file = "sdimaging.ms"

main_chunksize = {"time": 63}  # baseline, polarization
outfile = "sdimaging.ps.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel_mode="none",
    overwrite=True,
    main_chunksize=main_chunksize,
)

Output file:  sdimaging.ps.zarr
[2025-04-02 12:49:33,294]     INFO    viperlog:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2025-04-02 12:49:33,297]     INFO    viperlog:  Number of partitions: 1 
[2025-04-02 12:49:33,298]     INFO    viperlog:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61] 


## Processing Set

In [4]:
from xradio.measurement_set import open_processing_set

outfile = "sdimaging.ps.zarr"
ps_xdt = open_processing_set(ps_store=outfile)
ps_xdt.xr_ps.summary()

,name,intents,shape,polarization,scan_name,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,sdimaging_0,"[OBSERVE_TARGET_ON_SOURCE, FREQUENCY_SWITCH_SIG]","(3843, 1, 1024, 2)","[XX, YY]","[1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2,...",spw_0,[FLS3a_0],[FLS3a_0],[],"[fk5, 0h00m00.00s, 0d00m00.00s]",1.419395e+09,1.421893e+09


In [5]:
ms_xdt = ps_xdt["sdimaging_0"]
ms_xdt

<xarray.DataTree 'sdimaging_0'>
Group: /sdimaging_0
│   Dimensions:                     (time: 3843, antenna_name: 1, frequency: 1024,
│                                    polarization: 2)
│   Coordinates:
│     * antenna_name                (antenna_name) <U3 12B 'GBT'
│       field_name                  (time) <U27 415kB ...
│     * frequency                   (frequency) float64 8kB 1.419e+09 ... 1.422e+09
│     * polarization                (polarization) <U2 16B 'XX' 'YY'
│       scan_name                   (time) <U21 323kB '1' '1' '1' ... '61' '61' '61'
│       telescope_name              (antenna_name) <U3 12B ...
│     * time                        (time) float64 31kB 1.069e+09 ... 1.069e+09
│   Data variables:
│       EFFECTIVE_INTEGRATION_TIME  (time, antenna_name) float64 31kB ...
│       FLAG                        (time, antenna_name, frequency, polarization) bool 8MB ...
│       SPECTRUM                    (time, antenna_name, frequency, polarization) float32 31MB ...
│       TIME_CENTROID               (time, antenna_name) float64 31kB ...
│       WEIGHT                      (time, antenna_name, frequency, polarization) float32 31MB ...
│   Attributes:
│       creation_date:     2025-04-02T16:49:33.313114+00:00
│       creator:           {'software_name': 'xradio', 'version': '0.0.48'}
│       data_groups:       {'base': {'correlated_data': 'SPECTRUM', 'date': '2025...
│       observation_info:  {'intents': ['OBSERVE_TARGET_ON_SOURCE', 'FREQUENCY_SW...
│       processor_info:    {'sub_type': '', 'type': ''}
│       schema_version:    4.0.-9988
│       type:              spectrum
├── Group: /sdimaging_0/antenna_xds
│       Dimensions:                 (antenna_name: 1, cartesian_pos_label: 3,
│                                    receptor_label: 2)
│       Coordinates:
│         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
│           mount                   (antenna_name) <U6 24B ...
│           polarization_type       (antenna_name, receptor_label) <U1 8B ...
│         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
│           station                 (antenna_name) <U9 36B ...
│           telescope_name          (antenna_name) <U3 12B ...
│       Data variables:
│           ANTENNA_DISH_DIAMETER   (antenna_name) float64 8B ...
│           ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 24B ...
│           ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 16B ...
│       Attributes:
│           overall_telescope_name:  GBT
│           relocatable_antennas:    False
│           type:                    antenna
├── Group: /sdimaging_0/field_and_source_xds_base
│       Dimensions:                 (field_name: 1, sky_dir_label: 2, line_label: 1)
│       Coordinates:
│         * field_name              (field_name) <U27 108B 'FLS3a_0'
│         * line_label              (line_label) <U21 84B '0'
│           line_name               (field_name, line_label) <U1 4B ''
│         * sky_dir_label           (sky_dir_label) <U3 24B 'ra' 'dec'
│           source_name             (field_name) <U27 108B 'FLS3a_0'
│       Data variables:
│           FIELD_REFERENCE_CENTER  (field_name, sky_dir_label) float64 16B 0.0 0.0
│           LINE_REST_FREQUENCY     (field_name, line_label) float64 8B ...
│           LINE_SYSTEMIC_VELOCITY  (field_name, line_label) float64 8B ...
│           SOURCE_LOCATION         (field_name, sky_dir_label) float64 16B ...
│       Attributes:
│           type:     field_and_source
├── Group: /sdimaging_0/pointing_xds
│       Dimensions:              (time_pointing: 3843, antenna_name: 1,
│                                 local_sky_dir_label: 2)
│       Coordinates:
│         * local_sky_dir_label  (local_sky_dir_label) <U3 24B 'az' 'alt'
│         * time_pointing        (time_pointing) float64 31kB 4.575e+09 ... 4.575e+09
│       Data variables:
│           POINTING_BEAM        (time_pointing, antenna_name, local_sky_dir_label) fl

In [6]:
ms_xdt.antenna_xds

<xarray.DataTree 'antenna_xds'>
Group: /sdimaging_0/antenna_xds
    Dimensions:                 (time: 3843, antenna_name: 1, frequency: 1024,
                                 polarization: 2, cartesian_pos_label: 3,
                                 receptor_label: 2)
    Coordinates:
      * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
        mount                   (antenna_name) <U6 24B ...
        polarization_type       (antenna_name, receptor_label) <U1 8B ...
      * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
        station                 (antenna_name) <U9 36B ...
        telescope_name          (antenna_name) <U3 12B ...
    Inherited coordinates:
      * antenna_name            (antenna_name) <U3 12B 'GBT'
      * frequency               (frequency) float64 8kB 1.419e+09 ... 1.422e+09
      * polarization            (polarization) <U2 16B 'XX' 'YY'
      * time                    (time) float64 31kB 1.069e+09 ... 1.069e+09
    Data variables:
        ANTENNA_DISH_DIAMETER   (antenna_name) float64 8B ...
        ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 24B ...
        ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 16B ...
    Attributes:
        overall_telescope_name:  GBT
        relocatable_antennas:    False
        type:                    antenna

In [7]:
ms_xdt.weather_xds

<xarray.DataTree 'weather_xds'>
Group: /sdimaging_0/weather_xds
    Dimensions:         (time: 3843, antenna_name: 1, frequency: 1024,
                         polarization: 2, station_name: 1, time_weather: 1)
    Coordinates:
      * station_name    (station_name) <U9 36B 'GREENBANK'
      * time_weather    (time_weather) float64 8B 4.576e+09
    Inherited coordinates:
      * antenna_name    (antenna_name) <U3 12B 'GBT'
      * frequency       (frequency) float64 8kB 1.419e+09 1.419e+09 ... 1.422e+09
      * polarization    (polarization) <U2 16B 'XX' 'YY'
      * time            (time) float64 31kB 1.069e+09 1.069e+09 ... 1.069e+09
    Data variables:
        PRESSURE        (station_name, time_weather) float64 8B ...
        REL_HUMIDITY    (station_name, time_weather) float64 8B ...
        TEMPERATURE     (station_name, time_weather) float64 8B ...
        WIND_DIRECTION  (station_name, time_weather) float64 8B ...
        WIND_SPEED      (station_name, time_weather) float64 8B ...
    Attributes:
        type:     weather

In [8]:
ms_xdt.system_calibration_xds

<xarray.DataTree 'system_calibration_xds'>
Group: /sdimaging_0/system_calibration_xds
    Dimensions:               (time: 3843, antenna_name: 1, frequency: 1024,
                               polarization: 2, time_system_cal: 3843,
                               receptor_label: 2, frequency_system_cal: 1)
    Coordinates:
        antenna_id            (antenna_name) int32 4B ...
      * frequency_system_cal  (frequency_system_cal) int64 8B 0
        mount                 (antenna_name) <U6 24B ...
        polarization_type     (antenna_name, receptor_label) <U1 8B ...
      * receptor_label        (receptor_label) <U5 40B 'pol_0' 'pol_1'
        station               (antenna_name) <U9 36B ...
        telescope_name        (antenna_name) <U3 12B ...
      * time_system_cal       (time_system_cal) float64 31kB 4.575e+09 ... 4.575e+09
    Inherited coordinates:
      * antenna_name          (antenna_name) <U3 12B 'GBT'
      * frequency             (frequency) float64 8kB 1.419e+09 ... 1.422e+09
      * polarization          (polarization) <U2 16B 'XX' 'YY'
      * time                  (time) float64 31kB 1.069e+09 1.069e+09 ... 1.069e+09
    Data variables:
        TCAL                  (antenna_name, time_system_cal, receptor_label, frequency_system_cal) float64 61kB ...
        TSYS                  (antenna_name, time_system_cal, receptor_label, frequency_system_cal) float64 61kB ...
    Attributes:
        type:     system_calibration

In [9]:
ms_xdt.pointing_xds

<xarray.DataTree 'pointing_xds'>
Group: /sdimaging_0/pointing_xds
    Dimensions:              (time: 3843, antenna_name: 1, frequency: 1024,
                              polarization: 2, time_pointing: 3843,
                              local_sky_dir_label: 2)
    Coordinates:
      * local_sky_dir_label  (local_sky_dir_label) <U3 24B 'az' 'alt'
      * time_pointing        (time_pointing) float64 31kB 4.575e+09 ... 4.575e+09
    Inherited coordinates:
      * antenna_name         (antenna_name) <U3 12B 'GBT'
      * frequency            (frequency) float64 8kB 1.419e+09 ... 1.422e+09
      * polarization         (polarization) <U2 16B 'XX' 'YY'
      * time                 (time) float64 31kB 1.069e+09 1.069e+09 ... 1.069e+09
    Data variables:
        POINTING_BEAM        (time_pointing, antenna_name, local_sky_dir_label) float64 61kB ...
    Attributes:
        type:     pointing